# Introduction to Neural Networks: Perceptron and Multi-Layer Perceptron

In this tutorial, we will learn how to use Perceptron and Multi-Layer Perceptron (MLP) neural networks to solve simple classification tasks. We will walk through the necessary steps to implement and train these networks using PyTorch.

## Index
1. Imports
2. Models
    - Perceptron
    - Multi-Layer Perceptron
2. Data


## Imports

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim

## Models

### Perceptron

The Perceptron is one of the simplest types of artificial neural networks. It consists of a single layer of neurons, each with a set of weights and biases. The output of the Perceptron is calculated as the weighted sum of the inputs plus the bias, passed through an activation function.

$$z = \mathbf{w} \cdot \mathbf{x} + b = w_1 x_1 + w_2 x_2 + \ldots + w_n x_n + b$$

$$y = \gamma(z)$$

Where:

- $\mathbf{w}$ is the weight vector
- $\mathbf{x}$ is the input vector
- $b$ is the bias
- $\gamma$ is the sigmoid function.

To implement the perceptron in PyTorch, we can create a class that inherits from `torch.nn.Module` and the define the different elements of the network in the constructor and the forward pass in the `forward` method. You can use the matrix multiplication operation `torch.matmul` to calculate the weighted sum of the inputs and the bias.

In [ ]:
class Perceptron(nn.Module):
    def __init__(self, input_size, output_size):
        super(Perceptron, self).__init__()
        self.W = nn.Parameter(torch.randn(output_size, input_size) * 0.01)
        self.b = nn.Parameter(torch.zeros(output_size))

    def forward(self, x):
        x = torch.matmul(x, self.W.T) + self.b
        return x

### Multi-layer perceptron

THe multi-layer perceptron (MLP) is a type of feedforward neural network that consists of multiple layers of neurons. Each layer is fully connected to the next layer. The formula to calculate the output of an MLP is similar to the perceptron, but with the addition of non-linear activation functions between the layers:

$$z^{(1)} = \mathbf{W}^{(1)} \cdot \mathbf{x} + \mathbf{b}^{(1)}$$
$$h^{(1)} = \phi(z^{(1)})$$
$$z^{(2)} = \mathbf{W}^{(2)} \cdot \mathbf{h}^{(1)} + \mathbf{b}^{(2)}$$
$$y = \gamma(z^{(2)})$$

Where:
- x is the input vector.
- $\mathbf{W}^{(1)}$ and $\mathbf{W}^{(2)}$ are the weight matrices of the first and second layers.
- $\mathbf{b}^{(1)}$ and $\mathbf{b}^{(2)}$ are the bias vectors of the first and second layers.
- $\phi$ is the activation function. In this case, we will use the ReLU activation function.
- $\gamma$ is the output activation function. In this case, we will use the sigmoid activation function.


Note that if we don't include any non-linear activation functions, the MLP is equivalent to a linear regression model. However, by adding non-linear activation functions, the MLP can learn complex patterns in the data. 

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Initialize weights and biases for the first Linear layer
        self.W_ih = nn.Parameter(torch.randn(hidden_size, input_size) * 0.01)
        self.b_ih = nn.Parameter(torch.zeros(hidden_size))

        # Initialize weights and biases for the second Linear layer
        self.W_ho = nn.Parameter(torch.randn(output_size, hidden_size) * 0.01)
        self.b_ho = nn.Parameter(torch.zeros(output_size))

    def forward(self, x, non_linear_activation='relu'):
        if non_linear_activation=='relu':
            # Apply the first linear layer with ReLU activation
            h = torch.relu(torch.matmul(x, self.W_ih.T) + self.b_ih)
        elif non_linear_activation=='sigmoid':
            # Apply the first linear layer with Sigmoid activation
            h = torch.sigmoid(torch.matmul(x, self.W_ih.T) + self.b_ih)
        else:
            h = torch.matmul(x, self.W_ih.T) + self.b_ih

        # Apply the second linear layer
        x = torch.matmul(h, self.W_ho.T) + self.b_ho
        return x


## Data

In [ ]:
def plot_decision_boundary(X, Y, title, model=None):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))
    grid = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)
    if model is not None:
        with torch.no_grad():
            Z = model(grid)
            Z = torch.sigmoid(Z).numpy()  # Apply sigmoid to the output
            Z = (Z > 0.5).astype(int)
        Z = Z.reshape(xx.shape)
        plt.contourf(xx, yy, Z, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=Y, edgecolors='k', marker='o')
    plt.title(title)
    plt.show()
    
def generate_linear_data(N=100, D=2):
    X = np.random.randn(N, D)
    X[:N//2, :] += 1
    X[N//2:, :] -= 1
    Y = np.concatenate((np.zeros(N//2), np.ones(N//2)))
    X = torch.tensor(X, dtype=torch.float32)
    Y = torch.tensor(Y, dtype=torch.long)
    return X, Y

def generate_xor_data(n_points=100):
    # Base XOR points
    base_points = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
    base_labels = np.array([0, 1, 1, 0], dtype=np.float32)
    
    # Generate more points around the base points
    X = []
    Y = []
    for _ in range(n_points // 4):
        for point, label in zip(base_points, base_labels):
            noise = np.random.normal(0, 0.1, size=point.shape)
            X.append(point + noise)
            Y.append(label)
    
    X = np.array(X, dtype=np.float32)
    Y = np.array(Y, dtype=np.float32)
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y
# Task 1: Linearly Separate Two Clouds of Dots
X_linear, Y_linear = generate_linear_data()
input_size = 2
output_size = 1
plot_decision_boundary(X_linear, Y_linear, 'Linear Data')
X_xor, Y_xor = generate_xor_data()
plot_decision_boundary(X_xor, Y_xor, 'XOR Data')


## Training

In [ ]:
def train_model(model,  X, Y, num_epochs=1000, print_interval=100):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1)
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, Y.float().view(-1, 1))
        loss.backward()
        optimizer.step()
        if (epoch + 1) % print_interval == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


## Testing the models

In [ ]:
# Task 1: Linearly Separate Two Clouds of Dots
X_linear, Y_linear = generate_linear_data()
input_size = 2
output_size = 1

# Perceptron
model_perceptron = Perceptron(input_size, output_size)
train_model(model_perceptron, X_linear, Y_linear, num_epochs=1000, print_interval=100)
plot_decision_boundary(X_linear, Y_linear, 'Perceptron - Linear Data', model=model_perceptron)

# MLP
hidden_size = 2
model_mlp = MLP(input_size, hidden_size, output_size)
train_model(model_mlp, X_linear, Y_linear, num_epochs=1000, print_interval=100)
plot_decision_boundary(X_linear, Y_linear, 'MLP - Linear Data', model=model_mlp)

# Task 2: XOR Task
X_xor, Y_xor = generate_xor_data()

# Perceptron
model_perceptron = Perceptron(input_size, output_size)
train_model(model_perceptron, X_xor, Y_xor, num_epochs=10000, print_interval=1000)
plot_decision_boundary(X_xor, Y_xor, 'Perceptron - XOR Data', model=model_perceptron)

# MLP
model_mlp = MLP(input_size, hidden_size, output_size)
train_model(model_mlp, X_xor, Y_xor, num_epochs=10000, print_interval=1000)
plot_decision_boundary(X_xor, Y_xor, 'MLP - XOR Data', model=model_mlp)